In [1]:
from rdkit import Chem
import seaborn as sns

import pickle

In [2]:
file = 'dScribeSOAP_1_0 - QM9.pickle'

with open(file, 'rb') as handle:
    QM9 = pickle.load(handle)

QM9.drop(['Path to .mol'], axis=1, inplace=True)

QM9

,mol_id,Smiles-Canonical,Molecules-RDKit,SOAP - Sum,SOAP - Mean,h298
0,gdb_1,C,<rdkit.Chem.rdchem.Mol object at 0x000001EAFDD...,"[103.5042051711043, 38.42947623077812, 0.0, 14...","[20.70084103422086, 7.685895246155624, 0.0, 2....",-40.475117
1,gdb_3,O,<rdkit.Chem.rdchem.Mol object at 0x000001EAFDD...,"[32.768280815191176, 0.0, 18.85817268180666, 0...","[10.922760271730391, 0.0, 6.28605756060222, 0....",-76.400922
2,gdb_4,C#C,<rdkit.Chem.rdchem.Mol object at 0x000001EAFDD...,"[18.424646945289357, 21.394320190367054, 0.0, ...","[4.606161736322339, 5.3485800475917635, 0.0, 8...",-77.304583
3,gdb_6,C=O,<rdkit.Chem.rdchem.Mol object at 0x000001EAFDD...,"[26.521593496060945, 16.49801812803931, 7.4903...","[6.630398374015236, 4.124504532009827, 1.87258...",-114.479802
4,gdb_7,CC,<rdkit.Chem.rdchem.Mol object at 0x000001EAFDD...,"[146.92986599400263, 75.38243593931514, 0.0, 4...","[18.366233249250328, 9.422804492414393, 0.0, 5...",-79.759722
...,...,...,...,...,...,...
50631,gdb_123265,COCCOCCCO,<rdkit.Chem.rdchem.Mol object at 0x000001EA8C8...,"[284.2505507998751, 178.3987643687404, 57.4562...","[12.35871959999457, 7.756468016032191, 2.49809...",-462.518832
50632,gdb_123266,COCCOCCOC,<rdkit.Chem.rdchem.Mol object at 0x000001EA8B8...,"[286.90667227202675, 172.65231241884612, 58.54...","[12.474203142262033, 7.506622279080267, 2.5456...",-462.510857
50633,gdb_123267,OCCCCCCCO,<rdkit.Chem.rdchem.Mol object at 0x000001EA8B8...,"[342.07604168118513, 233.9475889972617, 38.254...","[13.683041667247405, 9.357903559890469, 1.5301...",-426.610273
50634,gdb_123268,OCCCCOCCO,<rdkit.Chem.rdchem.Mol object at 0x000001EA8B8...,"[279.32346719755935, 183.84576695954652, 57.07...","[12.144498573806928, 7.993294215632457, 2.4813...",-462.532004
